pick article  
split into sentences  
TF-IDF -> input  
score each sentences  
sort score in decending order  
print the top scoring sentences as the summary

In [2]:
raw = pd.read_csv('bbc_text_cls.csv')
raw.head()

,text,labels
0,Ad sales boost Time Warner profit\n\nQuarterly...,business
1,Dollar gains on Greenspan speech\n\nThe dollar...,business
2,Yukos unit buyer faces loan claim\n\nThe owner...,business
3,High fuel prices hit BA's profits\n\nBritish A...,business
4,Pernod takeover talk lifts Domecq\n\nShares in...,business


#### Use the first article as an example

#### General Function

In [105]:
import pandas as pd
import numpy as np
import textwrap
import nltk

from nltk import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

In [130]:
#help print the results more nicely
def wrap(x):
    return textwrap.fill(x, reshape_whitespace = False, fix_sentence_endings = True)

In [117]:
def non_zero_mean(np_arr):
    exist = (np_arr != 0)
    num = np_arr.sum(axis = 1)
    den = exist.sum(axis = 1)
    return num/den


tfidf = TfidfVectorizer(stop_words=stopwords.words('english'),
                        norm='l1') 
#normalization to elimilate the bias of long sentence
#as long sentences may contain more words and thus have higher scores

#i is the number of article in the dataset
def summary(i):
    article = raw.loc[i]['text']
    sentences = sent_tokenize(article.split('\n',1)[1]) #remove the title
    X = tfidf.fit_transform(sentences)
    ranking = pd.DataFrame(non_zero_mean(X))
    top_idx = (-ranking.loc[:, 0]).argsort()[0]
    print('Summary:', sentences[top_idx])
    #compare with title
    print('Title:', article.split('\n',1)[0])

In [118]:
summary(0)

Summary: But its own internet business, AOL, had has mixed fortunes.
Title: Ad sales boost Time Warner profit


In [119]:
summary(5)

Summary: The government was keen to play down the worrying implications of the data.
Title: Japan narrowly escapes recession


In [120]:
summary(25)

Summary: This is because of possible malfunctions with the braking systems.
Title: Safety alert as GM recalls cars


#### Compare with input library methods

In [121]:
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer

In [134]:
summarizer = TextRankSummarizer()
parser = PlaintextParser.from_string(
    raw.loc[0]['text'].split("\n",1)[1],
    Tokenizer("english"))
summary = summarizer(parser.document, sentences_count= 3)
print(summary)

(<Sentence: It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will try to sign up AOL's existing customers for high-speed broadband.>, <Sentence: But its film division saw profits slump 27% to $284m, helped by box-office flops Alexander and Catwoman, a sharp contrast to year-earlier, when the third and final film in the Lord of the Rings trilogy boosted results.>, <Sentence: It intends to adjust the way it accounts for a deal with German music publisher Bertelsmann's purchase of a stake in AOL Europe, which it had reported as advertising revenue.>)


In [135]:
summarizer = LsaSummarizer()
summary = summarizer(parser.document, sentences_count= 3)
print(summary)

(<Sentence: Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.>, <Sentence: Time Warner said on Friday that it now owns 8% of search-engine Google.>, <Sentence: For the full-year, TimeWarner posted a profit of $3.36bn, up 27% from its 2003 performance, while revenues grew 6.4% to $42.09bn.>)


In [139]:
pip show gensim #old version < 3.8 works

Name: gensim
Version: 4.2.0
Summary: Python framework for fast Vector Space Modelling
Home-page: http://radimrehurek.com/gensim
Author: Radim Rehurek
Author-email: me@radimrehurek.com
License: LGPL-2.1-only
Location: /opt/anaconda3/lib/python3.9/site-packages
Requires: scipy, smart-open, numpy
Required-by: 
Note: you may need to restart the kernel to use updated packages.
